# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0-rc1


# Dataset loading

In [4]:
dataset_name='pokemon'

In [5]:
images_dir = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
validation_percentage = 20
valid_format = 'png'

In [6]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [7]:
imgs_list = create_image_lists(
    image_dir=images_dir, 
    validation_pct=validation_percentage, 
    valid_imgae_formats=valid_format
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [8]:
inputs_shape= image_size=(200, 200, 3)
batch_size = 32//2
latents_dim = 32
intermediate_dim = 50

In [9]:
training_generator, testing_generator = get_generators(
    images_list=imgs_list, 
    image_dir=images_dir, 
    image_size=image_size, 
    batch_size=batch_size, 
    class_mode=None
)

  INFO     | Found 662 training files


  INFO     | Found 147 validation files


In [10]:
import tensorflow as tf

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: training_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: testing_generator, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((batch_size, ) + image_size)
)

In [12]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [13]:
_instance_scale

1.0

In [14]:
import numpy as np
from collections.abc import Iterable

In [15]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [16]:
_outputs_shape

120000

# Model's Layers definition

In [17]:
menc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

venc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [18]:
model_name = dataset_name+'VAE_Dense_reconst_ssmi'
experiments_dir='experiments'+sep_local+model_name

In [19]:
from training.autoencoding_basic.autoencoders.VAE import VAE as AE

In [20]:
inputs_shape=image_size

In [21]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    },
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [22]:
from utils.data_and_files.file_utils import create_if_not_exist

In [23]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [24]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonVAE_Dense_reconst_ssmi\\var_save_dir'

In [25]:
#to restore trained model, set filepath=_restore

In [26]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 16)      64        
_________________________________________________________________
dense_1 (Dense)              (None, 200, 200, 16)      272       
_________________________________________________________________
flatten (Flatten)            (None, 640000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                20480032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)             

  WARNING  | None


Model: "inference_logvariance"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_logvariance_inputs [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense_3 (Dense)              (None, 200, 200, 16)      64        
_________________________________________________________________
dense_4 (Dense)              (None, 200, 200, 16)      272       
_________________________________________________________________
flatten_1 (Flatten)          (None, 640000)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                20480032  
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)      

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_8 (Dense)              (None, 120000)            3960000   
_________________________________________________________________
reshape (Reshape)            (None, 200, 200, 3)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 200, 3)       12        
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 200, 3)       0

  WARNING  | None


In [27]:
from evaluation.quantitive_metrics.structural_similarity import prepare_ssim_multiscale
from statistical.losses_utilities import similarity_to_distance 

In [28]:
ae.compile(loss={'x_logits': similarity_to_distance(prepare_ssim_multiscale([ae.batch_size]+ae.get_inputs_shape()))})

Model: "pokemonVAE_Dense_reconst_ssmi"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (I [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
inference_logvariance (Model)   (None, 32)           20480496    inference_logvariance_inputs[0][0
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [(None, 32)]         0           inference_logvariance[1][0]      
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 32)]         0           tf_op_layer_mul[0][0]            
______________________________________________________________________

# Callbacks

In [29]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [30]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=12, 
    verbose=1, 
    restore_best_weights=False
)

In [31]:
ms = ModelSaver(filepath=_restore)

In [32]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonVAE_Dense_reconst_ssmi\\csv_dir\\pokemonVAE_Dense_reconst_ssmi.csv'

In [33]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [34]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

# Model Training

In [35]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=100,
    epochs=int(1e6), 
    verbose=1,
    callbacks=[ es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=10
)

Train for 10 steps, validate for 10 steps
Epoch 1/10
Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
10/10 [==============================] - ETA: 1:26 - loss: 0.1057 - x_logpdf_loss: -1.4174 - x_logits_loss: -0.1188 - z_latents_loss: 1.5545 - x_logits_psnr: 7.0561 - x_logits_ssim_multiscale: 1.2375 - x_logits_sharpdiff: 5.6524 - x_logits_mean_absolute_error: 0.4037 - x_logits_mean_squared_error: 0.198 - ETA: 39s - loss: 0.1068 - x_logpdf_loss: -1.4079 - x_logits_loss: -0.1419 - z_latents_loss: 1.5703 - x_logits_psnr: 7.0420 - x_logits_ssim_multiscale: 1.2838 - x_logits_sharpdiff: 5.7151 - x_logits_mean_absolute_error: 0.4044 - x_logits_mean_squared_error: 0.198 - ETA: 23s - loss: 0.1154 - x_logpdf_loss: -1.4108 - x_logits_loss: -0.1160 - z_latents_loss: 1.5531 - x_logits_psnr: 7.0900 - x_logits_ssim_multiscale: 1.2320 - x_logits_sharpdiff: 5.6067 - x_logits_mean_absolute_error: 0.4004 - x_logits_mean_squared_error: 

10/10 [==============================] - ETA: 2s - loss: -0.3254 - x_logpdf_loss: -1.4145 - x_logits_loss: -0.5917 - z_latents_loss: 1.5975 - x_logits_psnr: 7.1419 - x_logits_ssim_multiscale: 2.1834 - x_logits_sharpdiff: 6.4462 - x_logits_mean_absolute_error: 0.3996 - x_logits_mean_squared_error: 0.19 - ETA: 1s - loss: -0.3093 - x_logpdf_loss: -1.4200 - x_logits_loss: -0.6017 - z_latents_loss: 1.6129 - x_logits_psnr: 7.0583 - x_logits_ssim_multiscale: 2.2034 - x_logits_sharpdiff: 7.0452 - x_logits_mean_absolute_error: 0.4065 - x_logits_mean_squared_error: 0.19 - ETA: 1s - loss: -0.3492 - x_logpdf_loss: -1.4151 - x_logits_loss: -0.6298 - z_latents_loss: 1.5984 - x_logits_psnr: 7.0495 - x_logits_ssim_multiscale: 2.2596 - x_logits_sharpdiff: 6.8638 - x_logits_mean_absolute_error: 0.4066 - x_logits_mean_squared_error: 0.19 - ETA: 1s - loss: -0.3574 - x_logpdf_loss: -1.4111 - x_logits_loss: -0.6378 - z_latents_loss: 1.5986 - x_logits_psnr: 7.1880 - x_logits_ssim_multiscale: 2.2757 - x_logit

10/10 [==============================] - ETA: 2s - loss: -0.0890 - x_logpdf_loss: -1.3877 - x_logits_loss: -0.3610 - z_latents_loss: 1.5772 - x_logits_psnr: 7.0427 - x_logits_ssim_multiscale: 1.7219 - x_logits_sharpdiff: 6.9636 - x_logits_mean_absolute_error: 0.4066 - x_logits_mean_squared_error: 0.19 - ETA: 1s - loss: -0.3768 - x_logpdf_loss: -1.3887 - x_logits_loss: -0.6488 - z_latents_loss: 1.5678 - x_logits_psnr: 7.1844 - x_logits_ssim_multiscale: 2.2976 - x_logits_sharpdiff: 7.1871 - x_logits_mean_absolute_error: 0.3985 - x_logits_mean_squared_error: 0.19 - ETA: 1s - loss: -0.4083 - x_logpdf_loss: -1.3951 - x_logits_loss: -0.6650 - z_latents_loss: 1.5656 - x_logits_psnr: 7.2238 - x_logits_ssim_multiscale: 2.3301 - x_logits_sharpdiff: 7.1145 - x_logits_mean_absolute_error: 0.3977 - x_logits_mean_squared_error: 0.19 - ETA: 1s - loss: -0.4574 - x_logpdf_loss: -1.3969 - x_logits_loss: -0.7246 - z_latents_loss: 1.5766 - x_logits_psnr: 7.2433 - x_logits_ssim_multiscale: 2.4492 - x_logit

10/10 [==============================] - ETA: 2s - loss: -0.6018 - x_logpdf_loss: -1.3994 - x_logits_loss: -0.8805 - z_latents_loss: 1.6025 - x_logits_psnr: 7.7172 - x_logits_ssim_multiscale: 2.7610 - x_logits_sharpdiff: 7.2204 - x_logits_mean_absolute_error: 0.3772 - x_logits_mean_squared_error: 0.17 - ETA: 2s - loss: -0.6796 - x_logpdf_loss: -1.3941 - x_logits_loss: -0.9557 - z_latents_loss: 1.5943 - x_logits_psnr: 7.7657 - x_logits_ssim_multiscale: 2.9115 - x_logits_sharpdiff: 7.4820 - x_logits_mean_absolute_error: 0.3730 - x_logits_mean_squared_error: 0.16 - ETA: 1s - loss: -0.7273 - x_logpdf_loss: -1.3957 - x_logits_loss: -0.9953 - z_latents_loss: 1.5851 - x_logits_psnr: 7.6807 - x_logits_ssim_multiscale: 2.9907 - x_logits_sharpdiff: 7.4264 - x_logits_mean_absolute_error: 0.3773 - x_logits_mean_squared_error: 0.17 - ETA: 1s - loss: -0.7047 - x_logpdf_loss: -1.3962 - x_logits_loss: -0.9657 - z_latents_loss: 1.5766 - x_logits_psnr: 7.6247 - x_logits_ssim_multiscale: 2.9314 - x_logit

10/10 [==============================] - ETA: 2s - loss: -0.8831 - x_logpdf_loss: -1.3983 - x_logits_loss: -1.1212 - z_latents_loss: 1.5433 - x_logits_psnr: 7.4745 - x_logits_ssim_multiscale: 3.2424 - x_logits_sharpdiff: 8.0497 - x_logits_mean_absolute_error: 0.3856 - x_logits_mean_squared_error: 0.18 - ETA: 1s - loss: -0.9712 - x_logpdf_loss: -1.4000 - x_logits_loss: -1.2227 - z_latents_loss: 1.5600 - x_logits_psnr: 7.3956 - x_logits_ssim_multiscale: 3.4454 - x_logits_sharpdiff: 8.0513 - x_logits_mean_absolute_error: 0.3922 - x_logits_mean_squared_error: 0.18 - ETA: 1s - loss: -0.9653 - x_logpdf_loss: -1.4005 - x_logits_loss: -1.2305 - z_latents_loss: 1.5734 - x_logits_psnr: 7.5785 - x_logits_ssim_multiscale: 3.4610 - x_logits_sharpdiff: 7.8371 - x_logits_mean_absolute_error: 0.3822 - x_logits_mean_squared_error: 0.17 - ETA: 1s - loss: -0.9121 - x_logpdf_loss: -1.4029 - x_logits_loss: -1.1799 - z_latents_loss: 1.5776 - x_logits_psnr: 7.5646 - x_logits_ssim_multiscale: 3.3598 - x_logit

# Model Evaluation

## inception_score

In [36]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [37]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

calculating the inception_score mean ...


3it [02:42, 54.19s/it]

KeyboardInterrupt: 

## Frechet_inception_distance

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)